In [ ]:
import datetime

import numpy as np
import pandas as pd
import scipy as sp
import scipy.ndimage
import sklearn as sk
import sklearn.linear_model
from matplotlib import pyplot as plt

In [ ]:
eps = 0.00001
data_df = pd.read_csv("./2020-08-09_T7_restart_data.csv").transpose()[2:]
data = (
    pd.read_csv("./2020-08-09_T7_restart_data.csv").transpose()[2:].values.astype(float)
)
time = pd.read_csv("./2020-08-09_T7_restart_data.csv").transpose()[:1]
init_time = datetime.datetime.strptime("0:0:0", "%H:%M:%S")
t = np.array(
    [
        (datetime.datetime.strptime(item, "%H:%M:%S") - init_time).total_seconds()
        for item in time.values[0]
    ]
)
del_t = t[1:] - t[:-1]
del_OD = data[:, 1:] - data[:, :-1] + eps
log_del_OD = np.log2(del_OD)
log_del_OD[np.isnan(log_del_OD)] = -8
log_del_OD[log_del_OD < -8] = -8
log_del_OD = log_del_OD[:, 2:]
median_log_del_OD = sp.ndimage.median_filter(log_del_OD, size=(1, 5))
conditions = pd.read_csv("./conditions_key.csv").transpose()[1:]
# conditions.columns=["Media","Strain"]
# conditions = conditions.transpose()
# data = pd.concat([conditions,data]).transpose()

In [ ]:
above_thr = median_log_del_OD > -7.5
transition_up = np.where(above_thr[:, 1:] * (~above_thr[:, :-1]))

In [ ]:
first_above = {}
for i in range(len(transition_up[0])):
    row_num = transition_up[0][i]
    timept = transition_up[1][i]
    try:
        if timept < first_above[row_num]:
            first_above[row_num] = timept
    except:
        first_above[row_num] = timept

In [ ]:
plate_pos_list = data_df.index.tolist()
output_dict = {}
for row, timept in first_above.items():
    plate_pos = plate_pos_list[row]
    if median_log_del_OD[row, timept : timept + 10].shape[0] == 10:
        output_dict[plate_pos] = median_log_del_OD[row, timept : timept + 10]
output_df = pd.DataFrame(output_dict).transpose()
output_arr = output_df.values

In [ ]:
output_df.index.to_list()

In [ ]:
growth_rates = {}
df_plate_pos = output_df.index.to_list()
for i in range(len(df_plate_pos)):
    reg = sk.linear_model.LinearRegression().fit(
        np.linspace(0, 90, 10).reshape(-1, 1), output_arr[i]
    )
    slope = reg.coef_
    growth_rates[df_plate_pos[i]] = 1 / slope[0]
growth_rates

In [ ]:
slopes

In [ ]:
data_df.index.tolist()

In [ ]:
time_str = "13:55:26"
time_object = datetime.datetime.strptime(time_str, "%H:%M:%S").time()
print(type(time_object))
print(time_object)